<a href="https://colab.research.google.com/github/Vicky714/Vicky714/blob/main/SER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/msplabresearch/MSP-Podcast_Challenge_IS2025

Cloning into 'MSP-Podcast_Challenge_IS2025'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 47 (delta 11), reused 1 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 20.84 KiB | 10.42 MiB/s, done.
Resolving deltas: 100% (11/11), done.


[link text](https://)

In [8]:
%cd /content/MSP-Podcast_Challenge_IS2025
!bash run_cat.sh

/content/MSP-Podcast_Challenge_IS2025
{'Angry': 1.3415178571428572, 'Sad': 1.0152027027027026, 'Happy': 1.0016666666666667, 'Surprise': 0.9390625, 'Fear': 1.0152027027027026, 'Disgust': 0.9390625, 'Contempt': 1.0152027027027026, 'Neutral': 0.8536931818181818}
tensor([1.3415, 1.0152, 1.0017, 0.9391, 1.0152, 0.9391, 1.0152, 0.8537])
100% 601/601 [07:42<00:00,  1.30it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
100% 199/199 [00:48<00:00,  4.09it/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes 

In [12]:
import pandas as pd
import numpy as np

# Load the predictions CSV
predictions = pd.read_csv('/content/MSP-Podcast_Challenge_IS2025/model/cat_ser/7/results/Development.csv')

# Define the mapping of emotions to their abbreviations
emotion_map = ['A', 'S', 'H', 'U', 'F', 'D', 'C', 'N']

# Create a function to process a row
def get_emotion_class(row):
    # Convert the Prediction string to a list of floats
    scores = np.array(list(map(float, row['Prediction'].split(','))))
    # Find the index of the maximum value
    max_index = np.argmax(scores)
    # Print the .wav file name and the max index for debugging
    print(f"File: {row['FileName']}, Max Index: {max_index}")
    # Map the index to the emotion class
    return emotion_map[max_index]

# Apply the function to each row and create a new column for EmoClass
predictions['EmoClass'] = predictions.apply(get_emotion_class, axis=1)

# Select only the required columns and save to a new CSV
output_df = predictions[['FileName', 'EmoClass']]
output_df.to_csv('test3_predictions_wav2vec.csv', index=False)

print("Converted predictions saved to 'converted_predictions.csv'")


File: MSP-PODCAST_0052_0041.wav, Max Index: 1
File: MSP-PODCAST_0052_0100.wav, Max Index: 6
File: MSP-PODCAST_0058_0202.wav, Max Index: 1
File: MSP-PODCAST_0058_0383.wav, Max Index: 3
File: MSP-PODCAST_0133_0033.wav, Max Index: 5
File: MSP-PODCAST_0133_0382.wav, Max Index: 5
File: MSP-PODCAST_0133_0386.wav, Max Index: 4
File: MSP-PODCAST_0133_0917.wav, Max Index: 1
File: MSP-PODCAST_0133_1071.wav, Max Index: 5
File: MSP-PODCAST_0133_1141.wav, Max Index: 5
File: MSP-PODCAST_0157_0073.wav, Max Index: 5
File: MSP-PODCAST_0203_1052.wav, Max Index: 1
File: MSP-PODCAST_0261_0066.wav, Max Index: 6
File: MSP-PODCAST_0261_0131.wav, Max Index: 1
File: MSP-PODCAST_0261_0182.wav, Max Index: 7
File: MSP-PODCAST_0261_0298.wav, Max Index: 1
File: MSP-PODCAST_0261_0446.wav, Max Index: 1
File: MSP-PODCAST_0355_0293.wav, Max Index: 0
File: MSP-PODCAST_0357_0115.wav, Max Index: 1
File: MSP-PODCAST_0357_0166.wav, Max Index: 3
File: MSP-PODCAST_0380_0052_0001.wav, Max Index: 3
File: MSP-PODCAST_0380_0125_0

In [17]:
import pandas as pd

# Load the CSV files
true_labels = pd.read_csv('/content/filtered_original.csv', sep='\t')  # Tab-separated file
predictions = pd.read_csv('/content/MSP-Podcast_Challenge_IS2025/test3_predictions_wav2vec.csv', sep=',')  # Comma-separated file

# Strip any extra whitespace in column names
true_labels.columns = true_labels.columns.str.strip()
predictions.columns = predictions.columns.str.strip()

# Check column names in both DataFrames
print(f"True Labels Columns: {true_labels.columns.tolist()}")
print(f"Predictions Columns: {predictions.columns.tolist()}")

# Ensure 'FileName' and 'EmoClass' are present in both DataFrames
if 'FileName' not in true_labels.columns or 'FileName' not in predictions.columns:
    raise KeyError("Column 'FileName' not found in one or both CSV files.")

if 'EmoClass' not in true_labels.columns or 'EmoClass' not in predictions.columns:
    raise KeyError("Column 'EmoClass' not found in one or both CSV files.")

# Merge the true labels and predictions on 'FileName'
merged = pd.merge(predictions, true_labels[['FileName', 'EmoClass']], on='FileName', how='inner')

# Rename columns for clarity
merged.rename(columns={'EmoClass_x': 'Predicted', 'EmoClass_y': 'True'}, inplace=True)

# Compare predicted and true labels
merged['Correct'] = merged['Predicted'] == merged['True']

# Calculate overall accuracy
accuracy = merged['Correct'].mean()

# Output the accuracy
print(f"Overall Accuracy: {accuracy:.2%}")



True Labels Columns: ['FileName', 'EmoClass', 'EmoAct', 'EmoVal', 'EmoDom', 'SpkrID', 'Gender', 'Split_Set']
Predictions Columns: ['FileName', 'EmoClass']
Overall Accuracy: 9.55%
